**Engy-4390** Nuclear Systems Design and Analysis, UMass Lowell, Chem. Eng. Nuclear Program Fall 2024; Prof. V. F. de Almeida

# Discussion On Test One

**Student(s):**  Kyle Mills and Billy Manco and Donaita Anglade <br>
**Course:** Engy-4390: Nuclear Systems Design and Analysis  <br>
**Mentor and Collaborator:** Subash Sharma

UMass Lowell, Dept. of Chemical Engineering, Nuclear Energy Program 
<br>
**29 Sept 2024**

## [What is a Dirichet BC](#toc)<a id='Whatisit'></a>

The Dirichlet boundary condition (BC) is imposed on an ordinary or partial differental equation, such that the values that the solution takes along the boundary of the domain are fixed. The question of finding solutions to such equations is known as the DIrichlet problem, also called a fixed boundary condition.

THe applicaiton of a Dirichlet BC in the context of this project is as a heat transfer problem, where a surgace is held at a fixed temperature. 

The first test will be done with Dirichlet BC and using constant conductivity and source. The Dirichlet Boundary conditions are as follows:

\begin{align*}
T_{a}=20\; [°C]
\end{align*}

\begin{align*}
T_{b}=40\; [°C]
\end{align*}

Some parameters will be defined next. We define a constant value of $31 \left [ \frac{W}{mK} \right ] $ and a value of $156940\; \left [ \frac{W}{m^{3}} \right ]$ for the heat generation density